In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
import pandas as pd
from time import time

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load data
df=pd.read_csv('UNSW_2018_IoT_Botnet_Final_10_Best.csv',sep=";")  

del df['Unnamed: 0']
#df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,pkSeqID,proto,saddr,sport,daddr,dport,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack,category,subcategory
0,1,tcp,192.168.100.147,49960,192.168.100.7,80,9,0.068909,75,0.000000,1,0.068909,96,14.511893,0.566862,0.137818,1,DoS,HTTP
1,2,arp,192.168.100.7,-1,192.168.100.147,-1,10,0.000000,2,0.000131,2,0.000131,1,0.000000,0.000000,0.000131,1,DoS,HTTP
2,3,tcp,192.168.100.147,49962,192.168.100.7,80,11,0.064494,75,0.000000,1,0.064494,96,15.505319,0.567549,0.128988,1,DoS,HTTP
3,4,tcp,192.168.100.147,49964,192.168.100.7,80,12,0.064189,75,0.000000,1,0.064189,96,15.578993,0.567570,0.128378,1,DoS,HTTP
4,5,tcp,192.168.100.147,49966,192.168.100.7,80,13,0.063887,75,0.000000,1,0.063887,96,15.652637,0.567630,0.127774,1,DoS,HTTP


In [4]:
df = df.sample(frac=1)

df.shape

(3668522, 19)

In [5]:
df.columns = df.columns.to_series().apply(lambda x: x.strip())
df = df.reset_index(drop=True)

# General Information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3668522 entries, 0 to 3668521
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   pkSeqID            int64  
 1   proto              object 
 2   saddr              object 
 3   sport              object 
 4   daddr              object 
 5   dport              object 
 6   seq                int64  
 7   stddev             float64
 8   N_IN_Conn_P_SrcIP  int64  
 9   min                float64
 10  state_number       int64  
 11  mean               float64
 12  N_IN_Conn_P_DstIP  int64  
 13  drate              float64
 14  srate              float64
 15  max                float64
 16  attack             int64  
 17  category           object 
 18  subcategory        object 
dtypes: float64(6), int64(6), object(7)
memory usage: 531.8+ MB


# Exploratory Data Analysis

In [6]:
df.describe()

,pkSeqID,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,attack
count,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06
mean,1.834262e+06,1.213204e+05,8.871499e-01,8.253848e+01,1.017540e+00,3.134390e+00,2.231063e+00,9.245168e+01,4.455046e-01,2.955111e+00,3.020015e+00,9.998700e-01
std,1.059011e+06,7.579428e+04,8.037139e-01,2.439739e+01,1.483688e+00,1.186971e+00,1.517728e+00,1.817643e+01,6.028863e+01,7.245425e+02,1.860877e+00,1.140212e-02
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.171312e+05,5.487600e+04,3.001900e-02,6.900000e+01,0.000000e+00,3.000000e+00,1.819670e-01,1.000000e+02,0.000000e+00,1.559700e-01,2.806072e-01,1.000000e+00
50%,1.834262e+06,1.177690e+05,7.938960e-01,1.000000e+02,0.000000e+00,4.000000e+00,2.690125e+00,1.000000e+02,0.000000e+00,2.837830e-01,4.009111e+00,1.000000e+00
75%,2.751392e+06,1.849300e+05,1.745296e+00,1.000000e+02,2.151138e+00,4.000000e+00,3.565203e+00,1.000000e+02,0.000000e+00,4.882010e-01,4.293582e+00,1.000000e+00
max,3.668522e+06,2.622120e+05,2.496763e+00,1.000000e+02,4.980471e+00,1.100000e+01,4.981882e+00,1.000000e+02,5.882353e+04,1.000000e+06,4.999999e+00,1.000000e+00


# Extracting 10 best features

In [7]:
ten_best_features = df[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max','subcategory']]
#target_features = df[['attack','category','subcategory']]

# Data Preprocessing


In [8]:
# Label Encoding the target columns

le = LabelEncoder()
#target_features['category'] = le.fit_transform(target_features['category'])
ten_best_features['subcategory'] = le.fit_transform(ten_best_features['subcategory'])

ten_best_features.head()

,seq,stddev,N_IN_Conn_P_SrcIP,min,state_number,mean,N_IN_Conn_P_DstIP,drate,srate,max,subcategory
0,177897,0.644308,100,2.769445,4,3.679664,100,0.000000,0.725547,4.171223,7
1,56694,0.000000,68,0.000000,3,0.000000,68,0.000000,0.115974,0.000000,6
2,248412,0.036190,96,3.497858,4,3.549569,96,0.000000,0.280441,3.594149,7
3,223133,0.530751,100,2.541582,4,2.848723,100,0.000000,0.406295,3.909151,7
4,123564,0.068003,71,0.000000,1,0.039497,71,0.052861,0.130929,0.174314,6


# attack

In [9]:
from sklearn.model_selection import train_test_split

ten_best_features = ten_best_features.values

y = ten_best_features[:,10]
X = ten_best_features[:,0:10]

# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


# Scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
# number of class
len(np.unique(y_train))

8

# Extratrees Deep

In [11]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/subcategory.json")
gc = GCForest(config)

In [12]:
# X_enc is the concatenated predict_proba result of DeepExtraTrees
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2023-01-07 15:51:30,662][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 10)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 10)],y_test.shape=(1100557,)
[ 2023-01-07 15:51:30,779][cascade_classifier.fit_transform] group_dims=[10]
[ 2023-01-07 15:51:30,780][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 15:51:30,783][cascade_classifier.fit_transform] group_ends=[10]
[ 2023-01-07 15:51:30,785][cascade_classifier.fit_transform] X_train.shape=(2567965, 10),X_test.shape=(1100557, 10)
[ 2023-01-07 15:51:30,993][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 10), X_cur_test.shape=(1100557, 10)
[ 2023-01-07 15:52:57,731][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.91%
[ 2023-01-07 15:54:20,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 15:55:40,712][kfold_wrapper.log_eval_metrics] Accuracy

In [13]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of DeepExtratTrees = {:.6f} %".format(acc * 100))
 # Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of DeepExtratTrees = {:.6f} %".format(acc * 100))
 # Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[ 2023-01-07 16:05:10,775][cascade_classifier.transform] X_groups_test.shape=[(1100557, 10)]
[ 2023-01-07 16:05:10,805][cascade_classifier.transform] group_dims=[10]
[ 2023-01-07 16:05:10,807][cascade_classifier.transform] X_test.shape=(1100557, 10)
[ 2023-01-07 16:05:10,830][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1100557, 10)


Test Accuracy of DeepExtratTrees = 99.911499 %
[[     0      0      1      2      0      0      0      0]
 [     0    782      0      1      1      2      0      0]
 [     1      0     15      0      3      1      0      0]
 [     0      1      0    145      3      2      0      0]
 [     0      0      0      0   4840    533      1      0]
 [     0      0      0      2    404  21389      3      0]
 [     0      0      0      0      0      5 478079      3]
 [     0      0      0      1      0      0      4 594333]]




[ 2023-01-07 16:06:02,430][cascade_classifier.transform] X_groups_test.shape=[(1100557, 10)]
[ 2023-01-07 16:06:02,450][cascade_classifier.transform] group_dims=[10]
[ 2023-01-07 16:06:02,451][cascade_classifier.transform] X_test.shape=(1100557, 10)
[ 2023-01-07 16:06:02,487][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1100557, 10)


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      0.99      1.00       786
         2.0       0.94      0.75      0.83        20
         3.0       0.96      0.96      0.96       151
         4.0       0.92      0.90      0.91      5374
         5.0       0.98      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00      1.00    594338

    accuracy                           1.00   1100557
   macro avg       0.85      0.82      0.83   1100557
weighted avg       1.00      1.00      1.00   1100557



[ 2023-01-07 16:06:51,629][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 18)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 18)],y_test.shape=(1100557,)
[ 2023-01-07 16:06:51,795][cascade_classifier.fit_transform] group_dims=[18]
[ 2023-01-07 16:06:51,796][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 16:06:51,798][cascade_classifier.fit_transform] group_ends=[18]
[ 2023-01-07 16:06:51,799][cascade_classifier.fit_transform] X_train.shape=(2567965, 18),X_test.shape=(1100557, 18)
[ 2023-01-07 16:06:52,069][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 18), X_cur_test.shape=(1100557, 18)
[ 2023-01-07 16:07:54,500][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.90%
[ 2023-01-07 16:08:59,334][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.91%
[ 2023-01-07 16:09:59,427][kfold_wrapper.log_eval_metrics] Accuracy

Test Accuracy of DeepExtratTrees = 99.904503 %
[[     0      0      1      2      0      0      0      0]
 [     0    782      0      1      1      2      0      0]
 [     1      0     16      0      2      1      0      0]
 [     0      0      0    148      2      1      0      0]
 [     0      0      0      0   4840    534      0      0]
 [     0      0      0      1    492  21305      0      0]
 [     0      0      0      0      0      1 478084      2]
 [     0      0      0      1      0      0      6 594331]]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      0.99      1.00       786
         2.0       0.94      0.80      0.86        20
         3.0       0.97      0.98      0.97       151
         4.0       0.91      0.90      0.90      5374
         5.0       0.98      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00      1.00  

In [ ]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

Accuracy = []
def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of DeepExtratTrees = {:.6f} %".format(acc * 100),"Layer :",i)
    Accuracy.append(round(acc*100 , 2))
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))
    
# Deep
t = []
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    t.append(round(tt,2))
    print (" Time ",format(round(tt,3))," seconds")


[ 2023-01-07 16:18:17,884][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 24)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 24)],y_test.shape=(1100557,)
[ 2023-01-07 16:18:18,066][cascade_classifier.fit_transform] group_dims=[24]
[ 2023-01-07 16:18:18,067][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 16:18:18,068][cascade_classifier.fit_transform] group_ends=[24]
[ 2023-01-07 16:18:18,069][cascade_classifier.fit_transform] X_train.shape=(2567965, 24),X_test.shape=(1100557, 24)
[ 2023-01-07 16:18:18,340][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 24), X_cur_test.shape=(1100557, 24)
[ 2023-01-07 16:19:28,003][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.91%
[ 2023-01-07 16:20:45,167][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 16:22:01,044][kfold_wrapper.log_eval_metrics] Accuracy

Test Accuracy of DeepExtratTrees = 99.904503 % Layer : 0
[[     0      0      1      2      0      0      0      0]
 [     0    782      0      1      2      1      0      0]
 [     1      0     17      0      1      1      0      0]
 [     0      0      0    148      2      1      0      0]
 [     0      0      0      0   4807    567      0      0]
 [     0      0      0      4    457  21337      0      0]
 [     0      0      0      2      0      1 478084      0]
 [     0      0      0      1      0      0      6 594331]]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      0.99      1.00       786
         2.0       0.94      0.85      0.89        20
         3.0       0.94      0.98      0.96       151
         4.0       0.91      0.89      0.90      5374
         5.0       0.97      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00  

[ 2023-01-07 16:31:49,718][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 30)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 30)],y_test.shape=(1100557,)
[ 2023-01-07 16:31:49,956][cascade_classifier.fit_transform] group_dims=[30]
[ 2023-01-07 16:31:49,957][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 16:31:49,958][cascade_classifier.fit_transform] group_ends=[30]
[ 2023-01-07 16:31:49,959][cascade_classifier.fit_transform] X_train.shape=(2567965, 30),X_test.shape=(1100557, 30)
[ 2023-01-07 16:31:50,276][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 30), X_cur_test.shape=(1100557, 30)
[ 2023-01-07 16:33:15,200][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.91%
[ 2023-01-07 16:34:46,312][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 16:36:12,457][kfold_wrapper.log_eval_metrics] Accuracy

Test Accuracy of DeepExtratTrees = 99.902322 % Layer : 1
[[     0      0      1      2      0      0      0      0]
 [     0    783      0      0      2      1      0      0]
 [     0      0     17      0      2      1      0      0]
 [     0      0      0    149      2      0      0      0]
 [     0      0      0      0   4816    558      0      0]
 [     0      0      0      4    494  21300      0      0]
 [     0      0      0      1      0      1 478085      0]
 [     0      0      0      0      0      0      6 594332]]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      1.00      1.00       786
         2.0       0.94      0.85      0.89        20
         3.0       0.96      0.99      0.97       151
         4.0       0.91      0.90      0.90      5374
         5.0       0.97      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00  

[ 2023-01-07 16:46:57,468][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 36)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 36)],y_test.shape=(1100557,)
[ 2023-01-07 16:46:57,812][cascade_classifier.fit_transform] group_dims=[36]
[ 2023-01-07 16:46:57,813][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 16:46:57,814][cascade_classifier.fit_transform] group_ends=[36]
[ 2023-01-07 16:46:57,814][cascade_classifier.fit_transform] X_train.shape=(2567965, 36),X_test.shape=(1100557, 36)
[ 2023-01-07 16:46:58,311][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 36), X_cur_test.shape=(1100557, 36)
[ 2023-01-07 16:48:26,562][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.91%
[ 2023-01-07 16:50:02,906][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 16:51:42,780][kfold_wrapper.log_eval_metrics] Accuracy

...

[ 2023-01-07 17:03:52,992][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 42)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 42)],y_test.shape=(1100557,)
[ 2023-01-07 17:03:53,329][cascade_classifier.fit_transform] group_dims=[42]
[ 2023-01-07 17:03:53,339][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 17:03:53,340][cascade_classifier.fit_transform] group_ends=[42]
[ 2023-01-07 17:03:53,340][cascade_classifier.fit_transform] X_train.shape=(2567965, 42),X_test.shape=(1100557, 42)
[ 2023-01-07 17:03:53,909][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 42), X_cur_test.shape=(1100557, 42)
[ 2023-01-07 17:05:41,104][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.92%
[ 2023-01-07 17:07:36,802][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 17:09:25,069][kfold_wrapper.log_eval_metrics] Accuracy

Test Accuracy of DeepExtratTrees = 99.906411 % Layer : 3
[[     0      0      1      2      0      0      0      0]
 [     0    783      0      0      2      1      0      0]
 [     0      0     17      0      2      1      0      0]
 [     0      0      0    149      2      0      0      0]
 [     0      0      0      0   4829    545      0      0]
 [     0      0      0      4    459  21335      0      0]
 [     0      0      0      1      0      5 478081      0]
 [     0      0      0      0      0      0      5 594333]]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      1.00      1.00       786
         2.0       0.94      0.85      0.89        20
         3.0       0.96      0.99      0.97       151
         4.0       0.91      0.90      0.91      5374
         5.0       0.97      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00  

[ 2023-01-07 17:24:02,499][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 48)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 48)],y_test.shape=(1100557,)
[ 2023-01-07 17:24:02,874][cascade_classifier.fit_transform] group_dims=[48]
[ 2023-01-07 17:24:02,876][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 17:24:02,876][cascade_classifier.fit_transform] group_ends=[48]
[ 2023-01-07 17:24:02,879][cascade_classifier.fit_transform] X_train.shape=(2567965, 48),X_test.shape=(1100557, 48)
[ 2023-01-07 17:24:03,351][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 48), X_cur_test.shape=(1100557, 48)
[ 2023-01-07 17:25:44,166][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.92%
[ 2023-01-07 17:27:31,111][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 17:29:18,717][kfold_wrapper.log_eval_metrics] Accuracy

Test Accuracy of DeepExtratTrees = 99.906138 % Layer : 4
[[     0      0      1      2      0      0      0      0]
 [     0    783      0      0      2      1      0      0]
 [     0      0     17      0      2      1      0      0]
 [     0      0      0    149      2      0      0      0]
 [     0      0      0      0   4815    559      0      0]
 [     0      0      0      4    447  21347      0      0]
 [     0      0      0      0      0      5 478082      0]
 [     0      0      0      0      0      0      7 594331]]


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       1.00      1.00      1.00       786
         2.0       0.94      0.85      0.89        20
         3.0       0.96      0.99      0.97       151
         4.0       0.91      0.90      0.90      5374
         5.0       0.97      0.98      0.98     21798
         6.0       1.00      1.00      1.00    478087
         7.0       1.00      1.00  

[ 2023-01-07 17:42:52,174][cascade_classifier.fit_transform] X_groups_train.shape=[(2567965, 54)],y_train.shape=(2567965,),X_groups_test.shape=[(1100557, 54)],y_test.shape=(1100557,)
[ 2023-01-07 17:42:52,555][cascade_classifier.fit_transform] group_dims=[54]
[ 2023-01-07 17:42:52,556][cascade_classifier.fit_transform] group_starts=[0]
[ 2023-01-07 17:42:52,557][cascade_classifier.fit_transform] group_ends=[54]
[ 2023-01-07 17:42:52,558][cascade_classifier.fit_transform] X_train.shape=(2567965, 54),X_test.shape=(1100557, 54)
[ 2023-01-07 17:42:53,056][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2567965, 54), X_cur_test.shape=(1100557, 54)
[ 2023-01-07 17:44:42,688][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.92%
[ 2023-01-07 17:46:44,745][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.92%
[ 2023-01-07 17:48:47,042][kfold_wrapper.log_eval_metrics] Accuracy

In [ ]:
print ("Opt_layer_num =",Accuracy.index(max(Accuracy)), " Accuracy =", max(Accuracy) ,
       "Time =" , np.sum(t[0:Accuracy.index(max(Accuracy))+1]))

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=100)
plt.plot(Accuracy, '-o', ms=20, lw=2, alpha=0.7, mfc='orange')
plt.ylabel('Accuracy')
plt.xlabel('Layer numbers')
plt.show()

In [ ]:
Accuracy

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from deepforest import CascadeForestClassifier


model = CascadeForestClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("\nTesting Accuracy: {:.3f} %".format(acc))